### Square symmetries and the permutations of four elements in a square.

Some bits and pieces on the symmetries of a square and how these 
interact with the number of ways of arranging four elements at
the corners of a square

In [1]:
import numpy as np
import itertools

The operations that give the symmetries of a square, four rotations (including the identity rotation of zero degrees)
and four reflections.

Notation as used by http://www.cs.umb.edu/~eb/d4/


In [27]:
def r0(a): # Identity
    return a

def r1(a): # 90
    return np.rot90(a)

def r2(a): # 180
    return r1(r1(a))

def r3(a): # 270
    return r2(r1(a))

def m1(a): # LR
    return np.fliplr(a)

def m2(a): # UD
    return np.flipud(a)

def d1(a): # .T
    return np.transpose(a)

def d2(a): # off-diagonal transpose
    return m1(r1(a))

allOps = [r0, r1, r2, r3, m1, m2, d1, d2]

nOps = len(allOps)

Get all the permutations for four elements.
Reshape each one into a 2x2 array.
Store them all in a list


In [20]:
a = np.arange(4)

perms2x2 = []

for p in itertools.permutations(a):
    perms2x2.append(np.reshape(p,(2,2)))
    
nPerms2x2 = len(perms2x2)

In [21]:
def getIndexInPerms2x2(a):
    '''
    Given a 2x2 array with the elements 0,1,2,3 in some order, find the 
    index corresponding to it in the list perms2x2 (see above)
    '''
    func = lambda k: np.all(perms2x2[k] == a)
    N = len(perms2x2)
    x = list(map(func, range(N)))
    
    if not np.any(x):
        raise Exception('Cannot find array in perms2x2')
    
    k = 0
    while not x[k]:
        k += 1
    
    return k
    
    

----

In [31]:
# The 'base' array 
arr = np.reshape(np.arange(4), newshape=(2,2))

print('Base array')
print(arr)

Base array
[[0 1]
 [2 3]]


Apply every operation to the base array, then identify which of the set of permutations can be reached.

In [32]:
reachable = dict.fromkeys(range(nPerms2x2), [])

for op in allOps:
    res = op(arr)
    kRes = getIndexInPerms2x2(res)
    reachable[kRes] = reachable[kRes] + [op.__name__]
    
# for a,b in itertools.product(range(nOps), range(nOps)):
#     opA, opB = allOps[a], allOps[b]
#     res = opB(opA(arr))
#     kRes = getIndexInPerms2x2(res)
#     reachable[kRes] = reachable[kRes] + [ (opB.__name__, opA.__name__) ]
    

All the basic operations r0, r1, ..., etc. preserve the property 'diagonally opposite'.

The starting array 

`
0 1
2 3
`

Has 0 diagonally opposite 3, so all eight results of applying each of the operations will still have 0-3 on a diagonal (and 1-2 on the other)


In [33]:
print( 'Reachable permutations: ')

for k in reachable:
    if len(reachable[k]) == 0:
        continue
    print(k)
    print(perms2x2[k])
    print(reachable[k])
    print('---')

Reachable permutations: 
0
[[0 1]
 [2 3]]
['r0']
---
2
[[0 2]
 [1 3]]
['d1']
---
7
[[1 0]
 [3 2]]
['m1']
---
10
[[1 3]
 [0 2]]
['r1']
---
13
[[2 0]
 [3 1]]
['r3']
---
16
[[2 3]
 [0 1]]
['m2']
---
21
[[3 1]
 [2 0]]
['d2']
---
23
[[3 2]
 [1 0]]
['r2']
---


In [14]:
print('Unreachable permutations')
unreachable = []
for k in reachable:
    if len(reachable[k]) == 0:
        unreachable.append(perms2x2[k])
        
for u in unreachable:
    print(u)
    print('--')


Unreachable permutations
[[0 1]
 [3 2]]
--
[[0 2]
 [3 1]]
--
[[0 3]
 [1 2]]
--
[[0 3]
 [2 1]]
--
[[1 0]
 [2 3]]
--
[[1 2]
 [0 3]]
--
[[1 2]
 [3 0]]
--
[[1 3]
 [2 0]]
--
[[2 0]
 [1 3]]
--
[[2 1]
 [0 3]]
--
[[2 1]
 [3 0]]
--
[[2 3]
 [1 0]]
--
[[3 0]
 [1 2]]
--
[[3 0]
 [2 1]]
--
[[3 1]
 [0 2]]
--
[[3 2]
 [0 1]]
--


The 16 'unreachable' permutations fall into two classes depending on which elements are diagonally opposite.

`
0 1
3 2`                  

and 

`
0 2
3 1
`

These together with the original array:

`
0 1
2 3
`

form three classes - these are the 'orbits' defined by the action of the group of square symmetries on the set of permutations of four elements arranged in a square.


---

##### The stuff below looks into the effect of applying the operations to the first two dimensions of a rank four tensor.

---

In [ ]:
mn = [(0,0), (0,1), (1,0), (1,1)]

def print1(a):
    for m, n in mn:
        print(a[m,n,:,:])
def print2(a):
    for m, n in mn:
        print(a[:,:,m,n])
        

In [ ]:
def applyOpToFirstTwoDims(op, a):
    dim = a.ndim
    if dim < 3:
        raise Exception('array must be at least 3d')
        
    dimsForIter = a.shape[2:]

    dim0 = arr3.shape[0]
    dim1 = arr3.shape[1]
    ixArgsStart = [range(dim0), range(dim1)]

    newshape = np.concatenate([a.shape[:2], 
                               np.ones(shape=(dim-2,), dtype=np.int) ])
    
    dimsSet = list(map(lambda n: range(n), dimsForIter))
    # dimsSet = [range(2), range(3)]
    ds = list(itertools.product(*dimsSet))
    for d in ds:
        ixArgs = ixArgsStart + [[x] for x in d]
        ix = np.ix_(*ixArgs)
        b = op(a[ix].squeeze())
        a[ix] = np.reshape(b, newshape=newshape)



In [ ]:
arr = np.reshape(np.arange(36), newshape=(2,2,3,3))

In [ ]:
print1(arr)

In [ ]:
arr2 = np.transpose(arr)

In [ ]:
print2(arr2)

In [ ]:
arr3 = arr2.copy()
applyOpToFirstTwoDims(d2, arr3)
print2(arr3)

In [ ]:

d1


In [ ]:
arr3 = np.reshape(np.arange(4), newshape=(2,2))

In [ ]:
print('arr3\n', arr3)
print('r0\n', r0(arr3))
print('r1\n', r1(arr3))
print('r2\n', r2(arr3))
print('r3\n', r3(arr3))
print('m1\n', m1(arr3))
print('m2\n', m2(arr3))
print('d1\n', d1(arr3))
print('d2\n', d2(arr3))


In [ ]:
len(perms2x2)


In [ ]:
arr3


In [ ]:
getIndexInPerms2x2(r1(arr3))

In [ ]:
op = d2

print (op(arr3))
k = getIndexInPerms2x2(op(arr3))
print (k)
print (perms2x2[k])

In [ ]:
for k in range(24):
    print(k, perms2x2[k])

In [ ]:
itertools.product(range(8), range(8))